# Ingeniería de Datos en Python

## Laboratorio 2

Objetivos Especificos: 

- Conectarse a las diversas Bases de Datos (BD).
- Extraer informacion de una BD.
- Enviar los datos extraídos a otra BD.

A continuación, la conexión a SQL Server.

In [3]:
import pyodbc as po
import pandas as pd

sql_credenciales = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=OMARJCM\SQLSERVER20219;'
    'DATABASE=NORTHWND;'
    'UID=sa;'
    'PWD=inexcelsisdeo;'
)

sql_conexion = po.connect( sql_credenciales )
sql_cursor = sql_conexion.cursor()

In [4]:
sql = """
    SELECT ord.OrderID, cu.CustomerID, cu.CompanyName, ROUND( SUM( od.Quantity * (od.UnitPrice * (1 - od.Discount)) ), 2) valor_total
    FROM Orders ord INNER JOIN [Order Details] od ON (ord.OrderID = od.OrderID)
            INNER JOIN Customers cu ON (ord.CustomerID = cu.CustomerID)
    GROUP BY ord.OrderID, cu.CustomerID, cu.CompanyName
    ORDER BY valor_total DESC, cu.CustomerID
"""

sql_df = pd.read_sql(sql, sql_conexion)
print(sql_df)

     OrderID CustomerID                 CompanyName  valor_total
0      10865      QUICK                  QUICK-Stop     16387.50
1      10981      HANAR               Hanari Carnes     15810.00
2      11030      SAVEA          Save-a-lot Markets     12615.05
3      10889      RATTC  Rattlesnake Canyon Grocery     11380.00
4      10417      SIMOB               Simons bistro     11188.40
..       ...        ...                         ...          ...
825    10898      OCEAN      Océano Atlántico Ltda.        30.00
826    10767      SUPRD            Suprêmes délices        28.00
827    10586      REGGC          Reggiani Caseifici        23.80
828    10807      FRANS              Franchi S.p.A.        18.40
829    10782      CACTU  Cactus Comidas para llevar        12.50

[830 rows x 4 columns]


In [6]:
sql_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   OrderID      830 non-null    int64  
 1   CustomerID   830 non-null    object 
 2   CompanyName  830 non-null    object 
 3   valor_total  830 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 26.1+ KB


A continuación, se presenta la conexión a PostgreSQL.

In [15]:
import psycopg2 as ps

pg_credenciales = {
    'dbname':'northwind',
    'user':'postgres',
    'password':'inexcelsisdeo',
    'host':'localhost',
    'port':5432
}

pg_conexion = ps.connect( **pg_credenciales )
pg_cursor = pg_conexion.cursor()

Se crea una base de datos Northwind en PostresSQL y luego también se crea la siguiente tabla:

~~~~sql
CREATE TABLE IF NOT EXISTS public.facturado_clientes
(
    "OrderID" integer NOT NULL,
    "CustomerID" character varying(200) COLLATE pg_catalog."default" NOT NULL,
    "CompanyName" character varying(200) COLLATE pg_catalog."default",
    valor_total double precision,
    CONSTRAINT facturado_clientes_pkey PRIMARY KEY ("CustomerID", "OrderID")
)
~~~~

In [16]:
tuplas = [ tuple(x) for x in sql_df.to_numpy() ]

sql = """
INSERT INTO facturado_clientes ("OrderID", "CustomerID", "CompanyName", "valor_total") VALUES (%s, %s, %s, %s);
"""

pg_cursor.executemany(sql, tuplas)
pg_conexion.commit()

In [17]:
sql = """
    SELECT *
    FROM facturado_clientes
"""
pg_cursor.execute( sql )

for registro in pg_cursor:
    print( registro )

(10865, 'QUICK', 'QUICK-Stop', 16387.5)
(10981, 'HANAR', 'Hanari Carnes', 15810.0)
(11030, 'SAVEA', 'Save-a-lot Markets', 12615.05)
(10889, 'RATTC', 'Rattlesnake Canyon Grocery', 11380.0)
(10417, 'SIMOB', 'Simons bistro', 11188.4)
(10817, 'KOENE', 'Königlich Essen', 10952.84)
(10897, 'HUNGO', 'Hungry Owl All-Night Grocers', 10835.24)
(10479, 'RATTC', 'Rattlesnake Canyon Grocery', 10495.6)
(10540, 'QUICK', 'QUICK-Stop', 10191.7)
(10691, 'QUICK', 'QUICK-Stop', 10164.8)
(10515, 'QUICK', 'QUICK-Stop', 9921.3)
(10372, 'QUEEN', 'Queen Cozinha', 9210.9)
(10424, 'MEREP', 'Mère Paillarde', 9194.56)
(11032, 'WHITC', 'White Clover Markets', 8902.5)
(10514, 'ERNSH', 'Ernst Handel', 8623.45)
(10353, 'PICCO', 'Piccolo und mehr', 8593.28)
(10816, 'GREAL', 'Great Lakes Food Market', 8446.45)
(10360, 'BLONP', 'Blondesddsl père et fils', 7390.2)
(11017, 'ERNSH', 'Ernst Handel', 6750.0)
(10776, 'ERNSH', 'Ernst Handel', 6635.27)
(10607, 'SAVEA', 'Save-a-lot Markets', 6475.4)
(10895, 'ERNSH', 'Ernst Handel

In [18]:
pg_cursor.close()
pg_conexion.close()

sql_cursor.close()
sql_conexion.close()